# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [53]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [54]:
def Clean_NHL():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #The first thing we need to do is get rid of the [Notes] from the wiki html
    cities['NHL'] = cities['NHL'].str.replace(r'[[].*[]]','')
    cities['NFL'] = cities['NFL'].str.replace(r'[[].*[]]','')
    cities['MLB'] = cities['MLB'].str.replace(r'[[].*[]]','')
    cities['NBA'] = cities['NBA'].str.replace(r'[[].*[]]','')

    #Next, some lines have multiple teams on them so we need to extract that data
    #I made it a little bit more readable by spreading out the regex on 3 lines
    #In this case there was only 1 instance where a region had 3 teams, so i only had to use 3 lines. 
    #This should be iterable, should look into how. 
    Allteams1 = cities['NHL'].str.extract(r'(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams1['Metropolitan area'] = cities['Metropolitan area']
    Allteams2 = cities['NHL'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams2['Metropolitan area'] = cities['Metropolitan area']
    Allteams3 = cities['NHL'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams3['Metropolitan area'] = cities['Metropolitan area']
    df=pd.concat([Allteams1, Allteams2, Allteams3]).sort_index(axis=0)
    df['team'] = df['team'].apply(lambda x: np.nan if x == "" else x)
    #df['team'] = df['team'].str.extract(r'(?:[\w.]*\ )([\w.]*)|([\w.]*)')
    df.dropna(axis=0,inplace=True)
    df.drop_duplicates(subset=['team'], keep=False,inplace=True)
    df_return = pd.merge(df,cities,on='Metropolitan area')
    df_return.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    df_return['Population'] = pd.to_numeric(df_return['Population'])
    del df_return['NHL']
    del df_return['MLB']
    del df_return['NBA']
    del df_return['NFL']
    junction = df_return['team'].str.extract(r'(?:[\w.]*\ )([\w.]*)|([\w.]*)')
    df1 = junction[0]
    df2 = junction[1]
    teamJoin = pd.concat([df1, df2],join='outer')
    teamJoin.dropna(inplace=True)
    teamJoin.sort_index(ascending=True)
    df_return['Join'] = teamJoin
    return df_return

In [55]:
Clean_NHL()

,team,Metropolitan area,Population,Join
0,Rangers,New York City,20153634,Rangers
1,Islanders,New York City,20153634,Islanders
2,Devils,New York City,20153634,Devils
3,Kings,Los Angeles,13310447,Kings
4,Ducks,Los Angeles,13310447,Ducks
5,Sharks,San Francisco Bay Area,6657982,Sharks
6,Blackhawks,Chicago,9512999,Blackhawks
7,Stars,Dallas–Fort Worth,7233323,Stars
8,Capitals,"Washington, D.C.",6131977,Capitals
9,Flyers,Philadelphia,6070500,Flyers


In [56]:
def Clean_WL():
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df['team'] = nhl_df['team'].str.replace(r'[*]','')
    nhl_df['year'] = nhl_df['year'].apply(lambda x: int(2018) if x==2018 else np.nan)
    nhl_df.dropna(inplace=True)
    nhl_df['GP']=nhl_df['GP'].apply(lambda x: x if x.isnumeric() else np.nan)
    nhl_df.dropna(inplace=True)
    nhl_df['L']=pd.to_numeric(nhl_df['L'])
    nhl_df['W']=pd.to_numeric(nhl_df['W'])
    nhl_df = nhl_df[['team','W','L']]
    nhl_df['ratio'] = nhl_df['W']/(nhl_df['L']+nhl_df['W'])
    junction = nhl_df['team'].str.extract(r'(?:[\w.]*\ )(?:[\w.]*\ )([\w.]*)|(?:[\w.]*\ )([\w.]*)')
    df1 = junction[0]
    df2 = junction[1]
    teamJoin = pd.concat([df1, df2],join='outer')
    teamJoin.dropna(inplace=True)
    teamJoin.sort_index(ascending=True)
    nhl_df['Join'] = teamJoin
    return nhl_df

In [57]:
Clean_WL()

,team,W,L,ratio,Join
1,Tampa Bay Lightning,54,23,0.701299,Lightning
2,Boston Bruins,50,20,0.714286,Bruins
3,Toronto Maple Leafs,49,26,0.653333,Leafs
4,Florida Panthers,44,30,0.594595,Panthers
5,Detroit Red Wings,30,39,0.434783,Wings
6,Montreal Canadiens,29,40,0.420290,Canadiens
7,Ottawa Senators,28,43,0.394366,Senators
8,Buffalo Sabres,25,45,0.357143,Sabres
10,Washington Capitals,49,26,0.653333,Capitals
11,Pittsburgh Penguins,47,29,0.618421,Penguins


In [58]:
def nhl_correlation(): 
    df1 = Clean_NHL()
    df2 = Clean_WL()
    nhl_corr = pd.merge(df1,df2,'outer',on='Join')
    nhl_corr = nhl_corr.groupby('Metropolitan area').agg({'ratio':np.nanmean, 'Population':np.nanmean})
    
    population_by_region = nhl_corr['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nhl_corr['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

In [59]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [60]:
def Clean_NBA():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #The first thing we need to do is get rid of the [Notes] from the wiki html
    cities['NHL'] = cities['NHL'].str.replace(r'[[].*[]]','')
    cities['NFL'] = cities['NFL'].str.replace(r'[[].*[]]','')
    cities['MLB'] = cities['MLB'].str.replace(r'[[].*[]]','')
    cities['NBA'] = cities['NBA'].str.replace(r'[[].*[]]','')

    #Next, some lines have multiple teams on them so we need to extract that data
    #I made it a little bit more readable by spreading out the regex on 3 lines
    #In this case there was only 1 instance where a region had 3 teams, so i only had to use 3 lines. 
    #This should be iterable, should look into how. 
    Allteams1 = cities['NBA'].str.extract(r'(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams1['Metropolitan area'] = cities['Metropolitan area']
    Allteams2 = cities['NBA'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams2['Metropolitan area'] = cities['Metropolitan area']
    Allteams3 = cities['NBA'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams3['Metropolitan area'] = cities['Metropolitan area']
    df=pd.concat([Allteams1, Allteams2, Allteams3]).sort_index(axis=0)
    df['team'] = df['team'].apply(lambda x: np.nan if x == "" else x)
    #df['team'] = df['team'].str.extract(r'(?:[\w.]*\ )([\w.]*)|([\w.]*)')
    df.dropna(axis=0,inplace=True)
    df.drop_duplicates(subset=['team'], keep=False,inplace=True)
    df_return = pd.merge(df,cities,on='Metropolitan area')
    df_return.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    df_return['Population'] = pd.to_numeric(df_return['Population'])
    del df_return['NHL']
    del df_return['MLB']
    del df_return['NBA']
    del df_return['NFL']
    junction = df_return['team'].str.extract(r'(?:[\w.]*\ )([\w.]*)|([\w.]*)')
    df1 = junction[0]
    df2 = junction[1]
    teamJoin = pd.concat([df1, df2],join='outer')
    teamJoin.dropna(inplace=True)
    teamJoin.sort_index(ascending=True)
    df_return['Join'] = teamJoin
    return df_return

In [61]:
Clean_NBA()

,team,Metropolitan area,Population,Join
0,Knicks,New York City,20153634,Knicks
1,Nets,New York City,20153634,Nets
2,Lakers,Los Angeles,13310447,Lakers
3,Clippers,Los Angeles,13310447,Clippers
4,Warriors,San Francisco Bay Area,6657982,Warriors
5,Bulls,Chicago,9512999,Bulls
6,Mavericks,Dallas–Fort Worth,7233323,Mavericks
7,Wizards,"Washington, D.C.",6131977,Wizards
8,76ers,Philadelphia,6070500,76ers
9,Celtics,Boston,4794447,Celtics


In [62]:
nba_df=pd.read_csv("assets/nba.csv")

In [63]:
def Clean_NBAWL():
    nhl_df=pd.read_csv("assets/nba.csv")
    nhl_df['team'] = nhl_df['team'].str.replace(r'\*','')
    nhl_df['team'] = nhl_df['team'].str.replace(r'\s\(\d*\)','')
    nhl_df['year'] = nhl_df['year'].apply(lambda x: int(2018) if x==2018 else np.nan)
    nhl_df.dropna(inplace=True)
    #nhl_df['GP']=nhl_df['GP'].apply(lambda x: x if x.isnumeric() else np.nan)
    #nhl_df.dropna(inplace=True)
    nhl_df['L']=pd.to_numeric(nhl_df['L'])
    nhl_df['W']=pd.to_numeric(nhl_df['W'])
    nhl_df = nhl_df[['team','W','L']]
    nhl_df['ratio'] = nhl_df['W']/(nhl_df['L']+nhl_df['W'])
    junction = nhl_df['team'].str.extract(r'(?:[\w.]*\ )(?:[\w.]*\ )([\w.]*)|(?:[\w.]*\ )([\w.]*)')
    df1 = junction[0]
    df2 = junction[1]
    teamJoin = pd.concat([df1, df2],join='outer')
    teamJoin.dropna(inplace=True)
    teamJoin.sort_index(ascending=True)
    nhl_df['Join'] = teamJoin
    return nhl_df

In [64]:
Clean_NBAWL()

,team,W,L,ratio,Join
0,Toronto Raptors,59,23,0.719512,Raptors
1,Boston Celtics,55,27,0.670732,Celtics
2,Philadelphia 76ers,52,30,0.634146,76ers
3,Cleveland Cavaliers,50,32,0.609756,Cavaliers
4,Indiana Pacers,48,34,0.585366,Pacers
5,Miami Heat,44,38,0.536585,Heat
6,Milwaukee Bucks,44,38,0.536585,Bucks
7,Washington Wizards,43,39,0.524390,Wizards
8,Detroit Pistons,39,43,0.475610,Pistons
9,Charlotte Hornets,36,46,0.439024,Hornets


In [65]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    df1 = Clean_NBA()
    df2 = Clean_NBAWL()
    nba_corr = pd.merge(df1,df2,'outer',on='Join')
    nba_corr = nba_corr.groupby('Metropolitan area').agg({'ratio':np.nanmean, 'Population':np.nanmean})
    
    population_by_region = nba_corr['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nba_corr['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 30 teams being analysed for NBA"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

In [66]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [67]:
def Clean_MLB():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #The first thing we need to do is get rid of the [Notes] from the wiki html
    cities['NHL'] = cities['NHL'].str.replace(r'[[].*[]]','')
    cities['NFL'] = cities['NFL'].str.replace(r'[[].*[]]','')
    cities['MLB'] = cities['MLB'].str.replace(r'[[].*[]]','')
    cities['NBA'] = cities['NBA'].str.replace(r'[[].*[]]','')

    #Next, some lines have multiple teams on them so we need to extract that data
    #I made it a little bit more readable by spreading out the regex on 3 lines
    #In this case there was only 1 instance where a region had 3 teams, so i only had to use 3 lines. 
    #This should be iterable, should look into how. 
    Allteams1 = cities['MLB'].str.extract(r'(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams1['Metropolitan area'] = cities['Metropolitan area']
    Allteams2 = cities['MLB'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams2['Metropolitan area'] = cities['Metropolitan area']
    Allteams3 = cities['MLB'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams3['Metropolitan area'] = cities['Metropolitan area']
    df=pd.concat([Allteams1, Allteams2, Allteams3]).sort_index(axis=0)
    df['team'] = df['team'].apply(lambda x: np.nan if x == "" else x)
    #df['team'] = df['team'].str.extract(r'(?:[\w.]*\ )([\w.]*)|([\w.]*)')
    df.dropna(axis=0,inplace=True)
    df.drop_duplicates(subset=['team'], keep=False,inplace=True)
    df_return = pd.merge(df,cities,on='Metropolitan area')
    df_return.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    df_return['Population'] = pd.to_numeric(df_return['Population'])
    del df_return['NHL']
    del df_return['MLB']
    del df_return['NBA']
    del df_return['NFL']
    junction = df_return['team'].str.extract(r'([\w.]*\ [\w.]*)|([\w.]*)')
    df1 = junction[0]
    df2 = junction[1]
    teamJoin = pd.concat([df1, df2],join='outer')
    teamJoin.dropna(inplace=True)
    teamJoin.sort_index(ascending=True)
    df_return['Join'] = teamJoin
    df_return['Join'] = df_return['team'].str.replace(r'(\s)','')
    return df_return

In [68]:
Clean_MLB()

,team,Metropolitan area,Population,Join
0,Yankees,New York City,20153634,Yankees
1,Mets,New York City,20153634,Mets
2,Dodgers,Los Angeles,13310447,Dodgers
3,Angels,Los Angeles,13310447,Angels
4,Giants,San Francisco Bay Area,6657982,Giants
5,Athletics,San Francisco Bay Area,6657982,Athletics
6,Cubs,Chicago,9512999,Cubs
7,White Sox,Chicago,9512999,WhiteSox
8,Rangers,Dallas–Fort Worth,7233323,Rangers
9,Nationals,"Washington, D.C.",6131977,Nationals


In [69]:
def Clean_MLBWL():
    nhl_df=pd.read_csv("assets/mlb.csv")
    nhl_df['team'] = nhl_df['team'].str.replace(r'[\*]',"")
    nhl_df['team'] = nhl_df['team'].str.replace(r'\(\d*\)',"")
    nhl_df['team'] = nhl_df['team'].str.replace(r'[\xa0]',"")
    nhl_df['year'] = nhl_df['year'].apply(lambda x: int(2018) if x==2018 else np.nan)
    nhl_df.dropna(inplace=True)
    #nhl_df['GP']=nhl_df['GP'].apply(lambda x: x if x.isnumeric() else np.nan)
    #nhl_df.dropna(inplace=True)
    nhl_df['W-L%']=pd.to_numeric(nhl_df['W-L%'])
    nhl_df = nhl_df[['team','W-L%']]
    nhl_df['ratio'] = nhl_df['W-L%']
    #junction = nhl_df['team'].str.extract(r'(?:[\w.]*\ )(?:[\w.]*\ )([\w.]*)|(?:[\w.]*\ )([\w.]*)')
    #df1 = junction[0]
    #df2 = junction[1]
    #teamJoin = pd.concat([df1, df2],join='outer')
    #teamJoin.dropna(inplace=True)
    #teamJoin.sort_index(ascending=True)
    nhl_df['Join'] = nhl_df['team']
    nhl_df['Join'] = nhl_df['Join'].str.replace(r'\ Sox','Sox')
    nhl_df['Join'] = nhl_df['Join'].str.replace(r'\ Jay','Jay')
    #matches every word with a space after it meaning you can leave the team name unmatched!
    nhl_df['Join'] = nhl_df['Join'].str.replace(r'[\w.]* ','')
    return nhl_df

In [70]:
Clean_MLBWL()

,team,W-L%,ratio,Join
0,Boston Red Sox,0.667,0.667,RedSox
1,New York Yankees,0.617,0.617,Yankees
2,Tampa Bay Rays,0.556,0.556,Rays
3,Toronto Blue Jays,0.451,0.451,BlueJays
4,Baltimore Orioles,0.290,0.290,Orioles
5,Cleveland Indians,0.562,0.562,Indians
6,Minnesota Twins,0.481,0.481,Twins
7,Detroit Tigers,0.395,0.395,Tigers
8,Chicago White Sox,0.383,0.383,WhiteSox
9,Kansas City Royals,0.358,0.358,Royals


In [71]:
    df1 = Clean_MLB()
    df2 = Clean_MLBWL()
    mlb_corr = pd.merge(df1,df2,'outer',on='Join')
    mlb_corr = mlb_corr.groupby('Metropolitan area').agg({'W-L%':np.nanmean, 'Population':np.nanmean})
    mlb_corr

,W-L%,Population
Metropolitan area,,
Atlanta,0.556,5789700
Baltimore,0.290,2798886
Boston,0.667,4794447
Chicago,0.483,9512999
Cincinnati,0.414,2165139
Cleveland,0.562,2055612
Dallas–Fort Worth,0.414,7233323
Denver,0.558,2853077
Detroit,0.395,4297617


In [72]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    df1 = Clean_MLB()
    df2 = Clean_MLBWL()
    mlb_corr = pd.merge(df1,df2,'outer',on='Join')
    mlb_corr = mlb_corr.groupby('Metropolitan area').agg({'W-L%':np.nanmean, 'Population':np.nanmean})
    
    population_by_region = mlb_corr['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_corr['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [73]:
mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [74]:
def Clean_NFL():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #The first thing we need to do is get rid of the [Notes] from the wiki html
    cities['NHL'] = cities['NHL'].str.replace(r'[[].*[]]','')
    cities['NFL'] = cities['NFL'].str.replace(r'[[].*[]]','')
    cities['MLB'] = cities['MLB'].str.replace(r'[[].*[]]','')
    cities['NBA'] = cities['NBA'].str.replace(r'[[].*[]]','')

    #Next, some lines have multiple teams on them so we need to extract that data
    #I made it a little bit more readable by spreading out the regex on 3 lines
    #In this case there was only 1 instance where a region had 3 teams, so i only had to use 3 lines. 
    #This should be iterable, should look into how. 
    Allteams1 = cities['NFL'].str.extract(r'(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams1['Metropolitan area'] = cities['Metropolitan area']
    Allteams2 = cities['NFL'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams2['Metropolitan area'] = cities['Metropolitan area']
    Allteams3 = cities['NFL'].str.extract(r'(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?:[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)'
                                     '(?P<team>[A-Z0-9]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')
    #Add City names back into the data frame
    Allteams3['Metropolitan area'] = cities['Metropolitan area']
    df=pd.concat([Allteams1, Allteams2, Allteams3]).sort_index(axis=0)
    df['team'] = df['team'].apply(lambda x: np.nan if x == "" else x)
    #df['team'] = df['team'].str.extract(r'(?:[\w.]*\ )([\w.]*)|([\w.]*)')
    df.dropna(axis=0,inplace=True)
    df.drop_duplicates(subset=['team'], keep=False,inplace=True)
    df_return = pd.merge(df,cities,on='Metropolitan area')
    df_return.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    df_return['Population'] = pd.to_numeric(df_return['Population'])
    del df_return['NHL']
    del df_return['MLB']
    del df_return['NBA']
    del df_return['NFL']
    junction = df_return['team'].str.extract(r'([\w.]*\ [\w.]*)|([\w.]*)')
    df1 = junction[0]
    df2 = junction[1]
    teamJoin = pd.concat([df1, df2],join='outer')
    teamJoin.dropna(inplace=True)
    teamJoin.sort_index(ascending=True)
    df_return['Join'] = teamJoin
    df_return['Join'] = df_return['team'].str.replace(r'(\s)','')
    return df_return

In [75]:
Clean_NFL()

,team,Metropolitan area,Population,Join
0,Giants,New York City,20153634,Giants
1,Jets,New York City,20153634,Jets
2,Rams,Los Angeles,13310447,Rams
3,Chargers,Los Angeles,13310447,Chargers
4,49ers,San Francisco Bay Area,6657982,49ers
5,Raiders,San Francisco Bay Area,6657982,Raiders
6,Bears,Chicago,9512999,Bears
7,Cowboys,Dallas–Fort Worth,7233323,Cowboys
8,Redskins,"Washington, D.C.",6131977,Redskins
9,Eagles,Philadelphia,6070500,Eagles


In [76]:
def Clean_NFLWL():
    nhl_df=pd.read_csv("assets/nfl.csv")
    nhl_df['year'] = nhl_df['year'].apply(lambda x: int(2018) if x==2018 else np.nan)
    nhl_df.dropna(inplace=True)
    nhl_df['W-L%']=pd.to_numeric(nhl_df['W-L%'],errors='coerce')
    nhl_df.dropna(inplace=True)
    nhl_df['team'] = nhl_df['team'].str.replace(r'[\*]',"")
    nhl_df['team'] = nhl_df['team'].str.replace(r'[\+]',"")
    nhl_df = nhl_df[['team','W-L%']]
    nhl_df['ratio'] = nhl_df['W-L%']
    nhl_df['Join'] = nhl_df['team']
    nhl_df['Join'] = nhl_df['Join'].str.replace(r'[\w.]* ','')
    return nhl_df

In [77]:
Clean_NFLWL()

,team,W-L%,ratio,Join
1,New England Patriots,0.688,0.688,Patriots
2,Miami Dolphins,0.438,0.438,Dolphins
3,Buffalo Bills,0.375,0.375,Bills
4,New York Jets,0.250,0.250,Jets
6,Baltimore Ravens,0.625,0.625,Ravens
7,Pittsburgh Steelers,0.594,0.594,Steelers
8,Cleveland Browns,0.469,0.469,Browns
9,Cincinnati Bengals,0.375,0.375,Bengals
11,Houston Texans,0.688,0.688,Texans
12,Indianapolis Colts,0.625,0.625,Colts


In [78]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    df1 = Clean_NFL()
    df2 = Clean_NFLWL()
    nfl_corr = pd.merge(df1,df2,'outer',on='Join')
    nfl_corr = nfl_corr.groupby('Metropolitan area').agg({'W-L%':np.nanmean, 'Population':np.nanmean})
    
    population_by_region = nfl_corr['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_corr['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

In [79]:
nfl_correlation()

0.004282141436392957

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

______________________________________________
______________________________________________
______________________________________________

We need to retrieve all the win-loss data by metropolitan area then create a selection for each of the permeations that need to be checked (NHL/NFL, NHL/MLB, NHL/NBA, NFL/MLB, NFL/NBA, MLB/NBA, etc.). The ttest can be checked for each sequence. 

In [104]:
def nhl_data():
    df1 = Clean_NHL()
    df2 = Clean_WL()
    nhl_corr = pd.merge(df1,df2,'outer',on='Join')
    nhl_corr = nhl_corr.groupby('Metropolitan area').agg({'ratio':np.nanmean, 'Population':np.nanmean})
    nhl_corr['W-L%'] = nhl_corr['ratio']
    return nhl_corr[['W-L%']]

In [116]:
isinstance(nhl_data(), pd.DataFrame)

True

In [106]:
def nba_data():
    df1 = Clean_NBA()
    df2 = Clean_NBAWL()
    nba_corr = pd.merge(df1,df2,'outer',on='Join')
    nba_corr = nba_corr.groupby('Metropolitan area').agg({'ratio':np.nanmean, 'Population':np.nanmean})
    nba_corr['W-L%']=nba_corr['ratio']
    return nba_corr[['W-L%']]

In [117]:
isinstance(nba_data(), pd.DataFrame)

True

In [108]:
def mlb_data():
    df1 = Clean_MLB()
    df2 = Clean_MLBWL()
    mlb_corr = pd.merge(df1,df2,'outer',on='Join')
    mlb_corr = mlb_corr.groupby('Metropolitan area').agg({'W-L%':np.nanmean, 'Population':np.nanmean})
    return mlb_corr[['W-L%']]

In [118]:
isinstance(mlb_data(), pd.DataFrame)

True

In [110]:
def nfl_data():
    df1 = Clean_NFL()
    df2 = Clean_NFLWL()
    nfl_corr = pd.merge(df1,df2,'outer',on='Join')
    nfl_corr = nfl_corr.groupby('Metropolitan area').agg({'W-L%':np.nanmean, 'Population':np.nanmean})
    return nfl_corr[['W-L%']]

In [119]:
isinstance(nfl_data(), pd.DataFrame)

True

In [138]:
def sport_selector(sport):
    if sport =='NHL':
        return nhl_data()
    elif sport =='NFL':
        return nfl_data()
    elif sport =='MLB':
        return mlb_data()
    elif sport=='NBA':
        return nba_data()
    else:
        return None


In [148]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for l in sports:
        for m in sports:
            if l !=m:
                df=pd.merge(sport_selector(l), sport_selector(m), 'inner', on=['Metropolitan area'])
                p_values.loc[l, m]=stats.ttest_rel(df['W-L%_x'], df['W-L%_y'])[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [149]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937445,0.030318,0.803459
NBA,0.937445,NaN,0.022297,0.951629
NHL,0.030318,0.022297,NaN,0.000703
MLB,0.803459,0.951629,0.000703,NaN
